In [1]:
# import urllib.request
# urllib.request.urlretrieve('https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt', 'shopping.txt')

('shopping.txt', <http.client.HTTPMessage at 0x274e92b3e50>)

In [2]:
import pandas as pd
import numpy as np

raw = pd.read_table('shopping.txt', names=['rating', 'review'])

raw['label'] = np.where(raw['rating'] > 3, [1], 0) # 조건에 따라 새로운 열을 만들어준다. 3점 이상이면 1, 아니면 0


In [4]:
# 데이터 전처리
raw['review'] = raw['review'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '') # 1. 한글과 공백을 제외하고 모두 제거

# raw.isnull().sum() # 2. 결측치 확인

raw.drop_duplicates(subset=['review'], inplace=True) # 3. 중복값 제거
print(raw)

#bag of words # 4. 유니크한 단어들을 추출
유니크문자 = raw['review'].tolist()
유니크문자 = ''.join(유니크문자) 


C:\Users\Owner\AppData\Local\Temp\ipykernel_12788\531986262.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  raw['review'] = raw['review'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '') # 1. 한글과 공백을 제외하고 모두 제거


        rating                                             review  label
0            5                                            배공빠르고 굿      1
1            2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고      0
2            5  아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...      1
3            2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...      0
4            5                   민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ      1
...        ...                                                ...    ...
199995       2                                       장마라그런가 달지않아요      0
199996       5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...      1
199997       5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요      1
199998       5                                       넘이쁘고 쎄련되보이네요      1
199999       5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다      1

[199391 rows x 3 columns]
